In [ ]:
## Program is written and run in google colab with GPU support

## Written by  Tanvir

## install the gpu version of the tensorflow

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
## detect the GPU
#device_name = tf.test.gpu_device_name()
#if device_name != '/device:GPU:0':
#  raise SystemError('GPU device not found')
#print('Found GPU at: {}'.format(device_name))

In [ ]:

##only use at colab notebook
from google.colab import files
uploaded = files.upload()

In [ ]:
data = pd.read_csv('housing.csv')

In [46]:
data.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [ ]:
data=data[['housing_median_age','total_rooms','population','households','median_income','median_house_value']]

In [ ]:
## select the feature and target
y_val = data['median_house_value']
x_data=data.drop('median_house_value',1)


In [ ]:
## train test split
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x_data,y_val,test_size=0.3)

In [51]:
x_train.shape

(14448, 5)

In [ ]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result
  
  
## another normalizatoon

#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler()
#scaler.fit(x_train)




In [ ]:
x_train_norm = normalize(x_train)
#x_train_norm = scaler.transform(x_train)
x_test_norm = normalize(x_test)
#x_test_norm = scaler.transform(x_test)

In [ ]:

#x_train_norm=x_train_norm.drop(['longitude','latitude'],1)

In [57]:
#x_test_norm=x_test_norm.drop(['longitude','latitude'],1)
x_train_norm.head()

,housing_median_age,total_rooms,population,households,median_income
6037,0.980392,0.012615,0.010230,0.018254,0.062509
15532,0.196078,0.141004,0.064380,0.159349,0.221445
16610,0.274510,0.121013,0.068836,0.120539,0.159384
3477,0.450980,0.156239,0.075871,0.143726,0.483297
1020,0.470588,0.049468,0.025141,0.058050,0.231197


In [ ]:
#x_test_norm
#X_train = pd.DataFrame(data=x_train_norm,columns=x_train.columns,index=x_train.index)
#x_test_normc
#X_train = pd.DataFrame(data=x_test_norm,columns=x_test.columns,index=x_test.index)

In [ ]:
### making input place holder for all those feature
age=tf.feature_column.numeric_column('housing_median_age')
rooms=tf.feature_column.numeric_column('total_rooms')
#bedrooms=tf.feature_column.numeric_column('total_bedrooms')
pop=tf.feature_column.numeric_column('population')
households=tf.feature_column.numeric_column('households')
income=tf.feature_column.numeric_column('median_income')

In [ ]:
## input column list
feat_col = [age,rooms,pop,households,income]

In [ ]:
##now create the input func and feed all the data based on the input column
input_func = tf.estimator.inputs.pandas_input_fn(x=x_train_norm,y=y_train,batch_size=10,shuffle=False)

In [88]:
## now train with the column list that we created 
## we create a Densely connectly nural network
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_col) ##[6,6,6] means three layers and 6 perceptron each
model.train(input_fn=input_func,steps=20000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpurusx149', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff5f029c4e0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph wa

In [89]:
model1 = tf.estimator.LinearRegressor(feature_columns=feat_col)
model1.train(input_fn=input_func,steps=20000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpfs85bj3y', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff5f0ce0e10>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph wa

In [ ]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test_norm,batch_size=10,shuffle=False)

In [ ]:
## make the prediction and compare with the real y_test

predict_data = model1.predict(predict_input_func)

In [75]:
predict_data

<generator object EstimatorV2.predict at 0x7ff5f0e75468>

In [76]:
predict_data_list = list(predict_data)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/tmp1ckez96m/model.ckpt-1445
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [77]:
predict_data_list

[{'predictions': array([23.621378], dtype=float32)},
 {'predictions': array([31.453558], dtype=float32)},
 {'predictions': array([32.657135], dtype=float32)},
 {'predictions': array([24.95565], dtype=float32)},
 {'predictions': array([33.897614], dtype=float32)},
 {'predictions': array([24.512285], dtype=float32)},
 {'predictions': array([40.957245], dtype=float32)},
 {'predictions': array([32.17949], dtype=float32)},
 {'predictions': array([28.89039], dtype=float32)},
 {'predictions': array([24.344618], dtype=float32)},
 {'predictions': array([24.926033], dtype=float32)},
 {'predictions': array([36.058296], dtype=float32)},
 {'predictions': array([30.49189], dtype=float32)},
 {'predictions': array([32.82203], dtype=float32)},
 {'predictions': array([33.090607], dtype=float32)},
 {'predictions': array([29.351711], dtype=float32)},
 {'predictions': array([24.505402], dtype=float32)},
 {'predictions': array([27.315426], dtype=float32)},
 {'predictions': array([30.411903], dtype=float32)}

In [ ]:
## we gonna calculate the RMSE
final_pred = [] 

for pred in predict_data_list:
  final_pred.append(pred['predictions'])

In [79]:
final_pred

[array([23.621378], dtype=float32),
 array([31.453558], dtype=float32),
 array([32.657135], dtype=float32),
 array([24.95565], dtype=float32),
 array([33.897614], dtype=float32),
 array([24.512285], dtype=float32),
 array([40.957245], dtype=float32),
 array([32.17949], dtype=float32),
 array([28.89039], dtype=float32),
 array([24.344618], dtype=float32),
 array([24.926033], dtype=float32),
 array([36.058296], dtype=float32),
 array([30.49189], dtype=float32),
 array([32.82203], dtype=float32),
 array([33.090607], dtype=float32),
 array([29.351711], dtype=float32),
 array([24.505402], dtype=float32),
 array([27.315426], dtype=float32),
 array([30.411903], dtype=float32),
 array([28.74376], dtype=float32),
 array([34.018147], dtype=float32),
 array([25.611015], dtype=float32),
 array([31.762232], dtype=float32),
 array([29.966644], dtype=float32),
 array([33.24878], dtype=float32),
 array([29.017056], dtype=float32),
 array([29.153252], dtype=float32),
 array([29.395443], dtype=float32),

In [82]:
from sklearn.metrics import mean_squared_error as mse
mse(y_test,final_pred)**0.5

237680.56956864672

In [84]:
data.describe().transpose()

##mean in the medican_house_value 206855.816909 	

,count,mean,std,min,25%,50%,75%,max
housing_median_age,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
total_rooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
median_income,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
median_house_value,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [ ]:
# make a another input function
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test_norm,y=y_test,batch_size=10,shuffle=False)

In [92]:
model1.evaluate(input_fn=eval_input_func,steps=20000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-09T05:32:47Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpfs85bj3y/model.ckpt-1445
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-09-05:32:48
INFO:tensorflow:Saving dict for global step 1445: average_loss = 56492036000.0, global_step = 1445, label/mean = 208262.53, loss = 564191400000.0, prediction/mean = 29.427483
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1445: /tmp/tmpfs85bj3y/model.ckpt-1445


{'average_loss': 56492036000.0,
 'global_step': 1445,
 'label/mean': 208262.53,
 'loss': 564191400000.0,
 'prediction/mean': 29.427483}